In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import json
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


%matplotlib inline

In [2]:
path = '/Volumes/GoogleDrive/My Drive/wish_local/DSF-1436 Fusion Incrementality Analysis/'

In [10]:
df = pd.read_csv(path+'data/dsf1436_user_retention_by_cohort_country.csv')
df = df.sort_values(['cohort', 'shipping_country', 'bucket'])
df = df[df['bucket'].apply(lambda x: x != 'not_in_exp')]
df

,cohort,shipping_country,bucket,users0,users,repurchase_d7,repurchase_d30,repurchase_d60,repurchase_d90,repurchase_d120,repurchase_d150,repurchase_d180,repurchase_d210,repurchase_d240,repurchase_d270,repurchase_d300,repurchase_d330,repurchase_d360,pct_2nd_phurcase_days_180d,avg_2nd_phurcase_days_180d,sd_2nd_phurcase_days_180d,pct_2nd_phurcase_days_360d,avg_2nd_phurcase_days_360d,sd_2nd_phurcase_days_360d,pct_repurchase_in_180d,pct_repurchase_in_360d,avg_orders_30d,avg_orders_60d,avg_orders_90d,avg_orders_180d,avg_orders_360d,sd_orders_30d,sd_orders_60d,sd_orders_90d,sd_orders_180d,sd_orders_360d,avg_gmv_30d,avg_gmv_60d,avg_gmv_90d,avg_gmv_180d,avg_gmv_360d,sd_gmv_30d,sd_gmv_60d,sd_gmv_90d,sd_gmv_180d,sd_gmv_360d,time
0,2020-01,BR,ANDROID_CONTROL,3621,3621,0.107153,0.172880,0.140569,0.116819,0.103563,0.071251,0.065452,0.059100,0.048053,0.051919,0.067385,0.054681,0.045291,0.616957,20.896469,38.787826,0.658105,36.038444,71.009603,0.617233,0.658382,3.432477,3.992543,4.442695,5.257387,6.235294,4.047247,5.261901,6.228844,7.745256,9.925427,20.181169,24.100917,26.792345,32.089084,38.807859,43.599819,52.131016,57.187687,67.075414,82.618560,1628360845
1,2020-01,BR,ANDROID_SHOW,67316,67316,0.104492,0.165548,0.145374,0.111073,0.096634,0.068126,0.061620,0.055351,0.047388,0.047745,0.060907,0.050018,0.042798,0.606884,21.575010,39.282864,0.646711,36.563334,71.101819,0.606958,0.646785,3.427432,3.972042,4.446060,5.245915,6.131603,4.706873,5.891453,6.798428,8.314642,10.301526,20.125225,23.611225,26.412877,31.748550,38.029429,61.757257,67.361519,71.579211,80.713652,94.537366,1628360845
2,2020-01,BR,IOS_CONTROL,1189,1189,0.104289,0.149706,0.127839,0.103448,0.092515,0.061396,0.065601,0.047939,0.033642,0.025231,0.044575,0.036165,0.035324,0.611438,21.068058,39.634036,0.641716,31.809182,63.084950,0.611438,0.641716,3.777965,4.255677,4.650126,5.433137,6.185029,4.376459,5.116003,5.707951,6.828423,8.778332,25.040567,28.248926,30.618974,36.439349,42.427803,41.728099,48.238619,52.078389,66.901614,81.898086,1628360845
3,2020-01,BR,IOS_SHOW,23636,23636,0.095659,0.156795,0.139491,0.105729,0.090413,0.063928,0.056185,0.047639,0.040743,0.039389,0.043831,0.039431,0.037147,0.608521,19.998201,37.836892,0.638221,31.095096,63.426061,0.608690,0.638391,4.002708,4.630733,5.184041,6.064689,6.953884,6.411556,9.238182,11.226083,13.141011,15.451420,25.395182,29.765510,33.262033,39.707099,47.078006,51.842834,66.985471,76.996162,94.317489,118.169634,1628360845
5,2020-01,CA,ANDROID_CONTROL,1012,1012,0.129447,0.175889,0.139328,0.118577,0.094862,0.083004,0.058300,0.067194,0.061265,0.060277,0.049407,0.044466,0.053360,0.603755,15.004467,31.627091,0.627470,23.908089,55.589090,0.603755,0.627470,4.252964,5.176877,5.912055,7.369565,8.973320,6.879485,9.243486,10.965684,14.831990,18.702166,39.620645,49.438461,57.444236,74.881494,95.087440,73.158607,100.128991,119.152899,162.925374,213.682597,1628360845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,2021-06,MX,IOS_SHOW,2914,2914,0.075154,0.100549,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.384351,8.440486,12.313179,0.384351,8.440486,12.313179,0.385724,0.385724,4.336651,4.770075,4.770075,4.770075,4.770075,6.749380,7.914695,7.914695,7.914695,7.914695,43.891798,46.538786,46.538786,46.538786,46.538786,90.200030,94.975498,94.975498,94.975498,94.975498,1628360845
805,2021-06,US,ANDROID_CONTROL,6018,6018,0.120804,0.111499,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.522599,5.415354,9.730740,0.522599,5.415354,9.730740,0.522765,0.522765,5.266035,5.891825,5.891825,5.891825,5.891825,11.159015,12.908365,12.908365,12.908365,12.908365,67.653055,75.776255,75.776255,75.776255,75.776255,198.433513,217.468168,217.468168,217.468168,217.468168,1628360845
806,2021-06,US,ANDROID_SHOW,112079,112079,0.123

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cohort                      90 non-null     object 
 1   bucket                      90 non-null     object 
 2   users0                      90 non-null     int64  
 3   users                       90 non-null     int64  
 4   repurchase_d7               90 non-null     float64
 5   repurchase_d30              90 non-null     float64
 6   repurchase_d60              90 non-null     float64
 7   repurchase_d90              90 non-null     float64
 8   repurchase_d120             90 non-null     float64
 9   repurchase_d150             90 non-null     float64
 10  repurchase_d180             90 non-null     float64
 11  repurchase_d210             90 non-null     float64
 12  repurchase_d240             90 non-null     float64
 13  repurchase_d270             90 non-nu

# Repurchase % in x days

In [38]:
rp_cols = []
for x in df.columns:
    if 'repurchase_d' in x:
        rp_cols.append(x)
rp_cols  

['repurchase_d7',
 'repurchase_d30',
 'repurchase_d60',
 'repurchase_d90',
 'repurchase_d120',
 'repurchase_d150',
 'repurchase_d180',
 'repurchase_d210',
 'repurchase_d240',
 'repurchase_d270',
 'repurchase_d300',
 'repurchase_d330',
 'repurchase_d360']

In [39]:
rp = df.sort_values(['cohort', 'shipping_country', 'bucket'])\
    [['cohort', 'shipping_country', 'bucket', 'users']+rp_cols].copy(deep=True)
rp.head()

,cohort,shipping_country,bucket,users,repurchase_d7,repurchase_d30,repurchase_d60,repurchase_d90,repurchase_d120,repurchase_d150,repurchase_d180,repurchase_d210,repurchase_d240,repurchase_d270,repurchase_d300,repurchase_d330,repurchase_d360
0,2020-01,BR,ANDROID_CONTROL,3621,0.107153,0.172880,0.140569,0.116819,0.103563,0.071251,0.065452,0.059100,0.048053,0.051919,0.067385,0.054681,0.045291
1,2020-01,BR,ANDROID_SHOW,67316,0.104492,0.165548,0.145374,0.111073,0.096634,0.068126,0.061620,0.055351,0.047388,0.047745,0.060907,0.050018,0.042798
2,2020-01,BR,IOS_CONTROL,1189,0.104289,0.149706,0.127839,0.103448,0.092515,0.061396,0.065601,0.047939,0.033642,0.025231,0.044575,0.036165,0.035324
3,2020-01,BR,IOS_SHOW,23636,0.095659,0.156795,0.139491,0.105729,0.090413,0.063928,0.056185,0.047639,0.040743,0.039389,0.043831,0.039431,0.037147
5,2020-01,CA,ANDROID_CONTROL,1012,0.129447,0.175889,0.139328,0.118577,0.094862,0.083004,0.058300,0.067194,0.061265,0.060277,0.049407,0.044466,0.053360


## calculate diff

In [40]:
def show_control_diff(x, data, c, diff_pct=False):
    """
    x: row
    data: original dataframe
    c: column to calculate diff
    diff_pct: False is return absoluate diff, True is to return diff %
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        control_value = data[(data['cohort']==x['cohort']) & \
                             (data['shipping_country']==x['shipping_country']) & \
                             (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][c].iloc[0]
        show_value = x[c]
        if diff_pct:
            return (show_value - control_value) / control_value
        else:
            return show_value - control_value

In [77]:
for c in rp_cols:
    rp[c+'_diff'] = rp.apply(lambda x: show_control_diff(x, rp, c, diff_pct=False), axis=1)
    
temp = zip(rp_cols, [c+'_diff' for c in rp_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

rp[['cohort', 'shipping_country','bucket', 'users']+temp_cols]

,cohort,shipping_country,bucket,users,repurchase_d7,repurchase_d7_diff,repurchase_d30,repurchase_d30_diff,repurchase_d60,repurchase_d60_diff,repurchase_d90,repurchase_d90_diff,repurchase_d120,repurchase_d120_diff,repurchase_d150,repurchase_d150_diff,repurchase_d180,repurchase_d180_diff,repurchase_d210,repurchase_d210_diff,repurchase_d240,repurchase_d240_diff,repurchase_d270,repurchase_d270_diff,repurchase_d300,repurchase_d300_diff,repurchase_d330,repurchase_d330_diff,repurchase_d360,repurchase_d360_diff
0,2020-01,BR,ANDROID_CONTROL,3621,0.107153,NaN,0.172880,NaN,0.140569,NaN,0.116819,NaN,0.103563,NaN,0.071251,NaN,0.065452,NaN,0.059100,NaN,0.048053,NaN,0.051919,NaN,0.067385,NaN,0.054681,NaN,0.045291,NaN
1,2020-01,BR,ANDROID_SHOW,67316,0.104492,-0.002660,0.165548,-0.007333,0.145374,0.004805,0.111073,-0.005745,0.096634,-0.006929,0.068126,-0.003125,0.061620,-0.003832,0.055351,-0.003749,0.047388,-0.000665,0.047745,-0.004174,0.060907,-0.006478,0.050018,-0.004663,0.042798,-0.002493
2,2020-01,BR,IOS_CONTROL,1189,0.104289,NaN,0.149706,NaN,0.127839,NaN,0.103448,NaN,0.092515,NaN,0.061396,NaN,0.065601,NaN,0.047939,NaN,0.033642,NaN,0.025231,NaN,0.044575,NaN,0.036165,NaN,0.035324,NaN
3,2020-01,BR,IOS_SHOW,23636,0.095659,-0.008630,0.156795,0.007089,0.139491,0.011652,0.105729,0.002280,0.090413,-0.002102,0.063928,0.002532,0.056185,-0.009416,0.047639,-0.000300,0.040743,0.007101,0.039389,0.014158,0.043831,-0.000744,0.039431,0.003267,0.037147,0.001823
5,2020-01,CA,ANDROID_CONTROL,1012,0.129447,NaN,0.175889,NaN,0.139328,NaN,0.118577,NaN,0.094862,NaN,0.083004,NaN,0.058300,NaN,0.067194,NaN,0.061265,NaN,0.060277,NaN,0.049407,NaN,0.044466,NaN,0.053360,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,2021-06,MX,IOS_SHOW,2914,0.075154,-0.014589,0.100549,0.010805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
805,2021-06,US,ANDROID_CONTROL,6018,0.120804,NaN,0.111499,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
806,2021-06,US,ANDROID_SHOW,112079,0.123029,0.002225,0.110886,-0.000613,0.000232,0.000232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
807,2021-06,US,IOS_CONTROL,3743,0.080417,NaN,0.082554,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN


## proportion test

In [78]:
from statsmodels.stats.proportion import proportions_ztest
# https://www.statsmodels.org/devel/generated/statsmodels.stats.proportion.proportions_ztest.html

def show_control_prop_test(x, data, pct_col, nobs_col, is_count=False, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    pct_col: column name of numerator or the percentage calcualted
    nobs_col: column name of denominator
    is_count: False if pct_col is percentage, True if pct_col is the integer (numerator)
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        n_control = data[(data['cohort']==x['cohort']) & \
                        (data['shipping_country']==x['shipping_country']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][nobs_col].iloc[0]
        n_show = x[nobs_col]
        if is_count: 
            count_control = data[(data['cohort']==x['cohort']) & \
                                 (data['shipping_country']==x['shipping_country']) & \
                                 (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][pct_col].iloc[0]
            count_show = x[pct_col]
        else:
            count_control = int(round(data[(data['cohort']==x['cohort']) & \
                                     (data['shipping_country']==x['shipping_country']) & \
                                     (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][pct_col].iloc[0] * n_control,0))
            count_show = int(round(x[pct_col] * n_show, 0))
        
        stat, pval = proportions_ztest((count_control,count_show), (n_control,n_show), alternative=alternative)
        
        return round(pval, 2)

## Clean format

In [79]:
for c in rp_cols:
    rp[c+'_test'] = rp.apply(lambda x: \
         show_control_prop_test(x, rp, c, 'users', is_count=False, alternative='two-sided')
         , axis=1)
    rp[c+'_diff'] = rp.apply(lambda x: f"{round(x[c+'_diff']*100, 2)}" + \
                             ("" if np.isnan(x[c+'_diff']) else "%") + \
                             ('***' if x[c+'_test']<=0.05 else ''), axis=1)
    

temp = zip(rp_cols, [c+'_diff' for c in rp_cols])#, [c+'_test' for c in rp_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

rp[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols].to_clipboard()

/usr/local/lib/python3.9/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std


In [83]:
rp.shape

(648, 43)

In [82]:
def color_pval(x):
    if '*' in str(x):
        if '-' in str(x):
            return "background-color: pink"
        else:
            return "background-color: lightgreen"
    return "background-color: white"

rp[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols].style.applymap(color_pval)

# Retention 
for each cohort, avg number of days between the first purchase and second purchase, within 180/360d.

In [45]:
ret_cols = []
for x in df.columns:
    if '2nd_phurcase_days_' in x:
        ret_cols.append(x)
ret_cols  

['pct_2nd_phurcase_days_180d',
 'avg_2nd_phurcase_days_180d',
 'sd_2nd_phurcase_days_180d',
 'pct_2nd_phurcase_days_360d',
 'avg_2nd_phurcase_days_360d',
 'sd_2nd_phurcase_days_360d']

In [85]:
ret = df[df['bucket'].apply(lambda x: x!='not_in_exp')].sort_values(['cohort', 'bucket'])\
    [['cohort', 'shipping_country', 'bucket', 'users']+ret_cols].copy(deep=True)
ret.tail()

,cohort,shipping_country,bucket,users,pct_2nd_phurcase_days_180d,avg_2nd_phurcase_days_180d,sd_2nd_phurcase_days_180d,pct_2nd_phurcase_days_360d,avg_2nd_phurcase_days_360d,sd_2nd_phurcase_days_360d
788,2021-06,FR,IOS_SHOW,8141,0.296401,8.037786,11.780746,0.296401,8.037786,11.780746
793,2021-06,GB,IOS_SHOW,13168,0.315993,7.004355,11.061436,0.315993,7.004355,11.061436
798,2021-06,IT,IOS_SHOW,5532,0.298626,9.390173,12.680430,0.298626,9.390173,12.680430
803,2021-06,MX,IOS_SHOW,2914,0.384351,8.440486,12.313179,0.384351,8.440486,12.313179
808,2021-06,US,IOS_SHOW,69124,0.413171,6.384004,10.667211,0.413171,6.384004,10.667211


## Calculate diff

In [86]:
for c in ret_cols:
    ret[c+'_diff'] = ret.apply(lambda x: show_control_diff(x, ret, c, diff_pct=False), axis=1)
    
temp = zip(['avg_2nd_phurcase_days_180d', 'avg_2nd_phurcase_days_360d'], [c+'_diff' for c in ['avg_2nd_phurcase_days_180d', 'avg_2nd_phurcase_days_360d']])
temp_cols = [l1 for l2 in temp for l1 in l2]

ret[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols].head()

,cohort,shipping_country,bucket,users,avg_2nd_phurcase_days_180d,avg_2nd_phurcase_days_180d_diff,avg_2nd_phurcase_days_360d,avg_2nd_phurcase_days_360d_diff
0,2020-01,BR,ANDROID_CONTROL,3621,20.896469,NaN,36.038444,NaN
5,2020-01,CA,ANDROID_CONTROL,1012,15.004467,NaN,23.908089,NaN
10,2020-01,CL,ANDROID_CONTROL,1217,16.493354,NaN,24.359698,NaN
15,2020-01,DE,ANDROID_CONTROL,1726,18.324740,NaN,32.000693,NaN
20,2020-01,FR,ANDROID_CONTROL,2169,16.715204,NaN,30.397823,NaN


## t test

In [87]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# https://github.com/scipy/scipy/blob/v1.7.1/scipy/stats/stats.py#L5879-

from scipy.stats import distributions

def _unequal_var_ttest_denom(v1, n1, v2, n2):
    vn1 = v1 / n1
    vn2 = v2 / n2
    with np.errstate(divide='ignore', invalid='ignore'):
        dof = (vn1 + vn2)**2 / (vn1**2 / (n1 - 1) + vn2**2 / (n2 - 1))

    # If dof is undefined, variances are zero (assumes n1 > 0 & n2 > 0).
    # Hence it doesn't matter what dof is as long as it's not NaN.
    dof = np.where(np.isnan(dof), 1, dof)
    denom = np.sqrt(vn1 + vn2)
    return dof, denom

def _ttest_finish(dof, t, alternative):
    """Common code between all 3 t-test functions."""
    if alternative == 'less':
        prob = distributions.t.cdf(t, dof)
    elif alternative == 'greater':
        prob = distributions.t.sf(t, dof)
    elif alternative == 'two-sided':
        prob = 2 * distributions.t.sf(np.abs(t), dof)
    else:
        raise ValueError("alternative must be "
                         "'less', 'greater' or 'two-sided'")
    return t, prob

def _ttest_ind_from_stats(mean1, mean2, denom, dof, alternative):

    d = mean1 - mean2
    with np.errstate(divide='ignore', invalid='ignore'):
        t = np.divide(d, denom)
    t, prob = _ttest_finish(dof, t, alternative)

    return (t, prob)



def unequal_var_ttest(v1, v2, n1, n2, m1, m2, alternative='two-sided'):
    dof, denom = _unequal_var_ttest_denom(v1, n1, v2, n2)
    t, pval = _ttest_ind_from_stats(m1, m2, denom, dof, alternative)
    
    return t, pval

In [88]:


def show_control_t_test(x, data, avg_col, std_col, n_col, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        v_control = data[(data['cohort']==x['cohort']) & \
                        (data['shipping_country']==x['shipping_country']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][std_col].iloc[0] ** 2
        v_show = x[std_col] ** 2
        
        n_control = data[(data['cohort']==x['cohort']) & \
                        (data['shipping_country']==x['shipping_country']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][n_col].iloc[0]
        n_show = x[n_col]
        
        m_control = data[(data['cohort']==x['cohort']) & \
                        (data['shipping_country']==x['shipping_country']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][avg_col].iloc[0]
        m_show = x[avg_col]
        
        t, pval = unequal_var_ttest(v_control, v_show, n_control, n_show, m_control, m_show, alternative)
        
        return round(pval, 4)


In [89]:
ret['users_2nd_phurcase_days_180d'] = (ret['pct_2nd_phurcase_days_180d'] * ret['users']).apply(lambda x: round(x, 0)).astype(int)
ret['users_2nd_phurcase_days_360d'] = (ret['pct_2nd_phurcase_days_360d'] * ret['users']).apply(lambda x: round(x, 0)).astype(int)

for c in ['2nd_phurcase_days_180d', '2nd_phurcase_days_360d']:
    ret[c+'_pval'] = ret.apply(lambda x: \
                                 show_control_t_test(x, data=ret, \
                                                     avg_col='avg_'+c, \
                                                     std_col='sd_'+c, \
                                                     n_col='users_'+c, \
                                                     alternative='two-sided') , axis=1)

    ret['avg_'+c+'_diff2'] = ret.apply(lambda x: f"{round(x['avg_'+c+'_diff'], 2)}" + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip(['avg_2nd_phurcase_days_180d', 'avg_2nd_phurcase_days_360d'], ['avg_'+c+'_diff2' for c in ['2nd_phurcase_days_180d', '2nd_phurcase_days_360d']])#, [c+'_test' for c in ret_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

ret[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols]

,cohort,shipping_country,bucket,users,avg_2nd_phurcase_days_180d,avg_2nd_phurcase_days_180d_diff2,avg_2nd_phurcase_days_360d,avg_2nd_phurcase_days_360d_diff2
0,2020-01,BR,ANDROID_CONTROL,3621,20.896469,nan,36.038444,nan
5,2020-01,CA,ANDROID_CONTROL,1012,15.004467,nan,23.908089,nan
10,2020-01,CL,ANDROID_CONTROL,1217,16.493354,nan,24.359698,nan
15,2020-01,DE,ANDROID_CONTROL,1726,18.324740,nan,32.000693,nan
20,2020-01,FR,ANDROID_CONTROL,2169,16.715204,nan,30.397823,nan
...,...,...,...,...,...,...,...,...
788,2021-06,FR,IOS_SHOW,8141,8.037786,1.0,8.037786,1.0
793,2021-06,GB,IOS_SHOW,13168,7.004355,-0.15,7.004355,-0.15
798,2021-06,IT,IOS_SHOW,5532,9.390173,1.0,9.390173,1.0
803,2021-06,MX,IOS_SHOW,2914,8.440486,1.08,8.440486,1.08


In [90]:
ret[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols].style.applymap(color_pval)

,cohort,shipping_country,bucket,users,avg_2nd_phurcase_days_180d,avg_2nd_phurcase_days_180d_diff2,avg_2nd_phurcase_days_360d,avg_2nd_phurcase_days_360d_diff2
0,2020-01,BR,ANDROID_CONTROL,3621,20.896469,nan,36.038444,nan
5,2020-01,CA,ANDROID_CONTROL,1012,15.004467,nan,23.908089,nan
10,2020-01,CL,ANDROID_CONTROL,1217,16.493354,nan,24.359698,nan
15,2020-01,DE,ANDROID_CONTROL,1726,18.324740,nan,32.000693,nan
20,2020-01,FR,ANDROID_CONTROL,2169,16.715204,nan,30.397823,nan
25,2020-01,GB,ANDROID_CONTROL,2597,12.589319,nan,18.473988,nan
30,2020-01,IT,ANDROID_CONTROL,1460,18.511861,nan,30.448349,nan
35,2020-01,MX,ANDROID_CONTROL,2459,14.263418,nan,20.557415,nan
40,2020-01,US,ANDROID_CONTROL,9575,12.231833,nan,17.504451,nan
1,2020-01,BR,ANDROID_SHOW,67316,21.575010,0.68,36.563334,0.52


# Not churn rate (repurchased in 180/360d)
for each cohort, % users who never repurchase again within 180d/360d

In [52]:
nc_cols = []
for x in df.columns:
    if 'ct_repurchase_in_' in x:
        nc_cols.append(x)
nc_cols  

['pct_repurchase_in_180d', 'pct_repurchase_in_360d']

In [61]:
nc = df[df['bucket'].apply(lambda x: x!='not_in_exp')].sort_values(['cohort', 'bucket'])\
    [['cohort', 'shipping_country', 'bucket', 'users']+nc_cols].copy(deep=True)
nc.head()

,cohort,shipping_country,bucket,users,pct_repurchase_in_180d,pct_repurchase_in_360d
0,2020-01,BR,ANDROID_CONTROL,3621,0.617233,0.658382
5,2020-01,CA,ANDROID_CONTROL,1012,0.603755,0.627470
10,2020-01,CL,ANDROID_CONTROL,1217,0.535744,0.553821
15,2020-01,DE,ANDROID_CONTROL,1726,0.537080,0.568366
20,2020-01,FR,ANDROID_CONTROL,2169,0.553711,0.585984


## calculate diff

In [62]:
for c in nc_cols:
    nc[c+'_diff'] = nc.apply(lambda x: show_control_diff(x, nc, c, diff_pct=False), axis=1)
    
temp = zip(nc_cols, [c+'_diff' for c in nc_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

nc[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols]

,cohort,shipping_country,bucket,users,pct_repurchase_in_180d,pct_repurchase_in_180d_diff,pct_repurchase_in_360d,pct_repurchase_in_360d_diff
0,2020-01,BR,ANDROID_CONTROL,3621,0.617233,NaN,0.658382,NaN
5,2020-01,CA,ANDROID_CONTROL,1012,0.603755,NaN,0.627470,NaN
10,2020-01,CL,ANDROID_CONTROL,1217,0.535744,NaN,0.553821,NaN
15,2020-01,DE,ANDROID_CONTROL,1726,0.537080,NaN,0.568366,NaN
20,2020-01,FR,ANDROID_CONTROL,2169,0.553711,NaN,0.585984,NaN
...,...,...,...,...,...,...,...,...
788,2021-06,FR,IOS_SHOW,8141,0.296892,0.001847,0.296892,0.001847
793,2021-06,GB,IOS_SHOW,13168,0.316373,0.005706,0.316373,0.005706
798,2021-06,IT,IOS_SHOW,5532,0.298988,-0.012431,0.298988,-0.012431
803,2021-06,MX,IOS_SHOW,2914,0.385724,-0.005302,0.385724,-0.005302


## proportion test

In [63]:
for c in nc_cols:
    nc[c+'_test'] = nc.apply(lambda x: \
         show_control_prop_test(x, nc, c, 'users', is_count=False, alternative='two-sided')
         , axis=1)
    nc[c+'_diff'] = nc.apply(lambda x: f"{round(x[c+'_diff']*100, 2)}" + \
                             ("" if np.isnan(x[c+'_diff']) else "%") + \
                             ('***' if x[c+'_test']<=0.05 else ''), axis=1)
    

temp = zip(nc_cols, [c+'_diff' for c in nc_cols])#, [c+'_test' for c in nc_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

nc[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols]

,cohort,shipping_country,bucket,users,pct_repurchase_in_180d,pct_repurchase_in_180d_diff,pct_repurchase_in_360d,pct_repurchase_in_360d_diff
0,2020-01,BR,ANDROID_CONTROL,3621,0.617233,nan,0.658382,nan
5,2020-01,CA,ANDROID_CONTROL,1012,0.603755,nan,0.627470,nan
10,2020-01,CL,ANDROID_CONTROL,1217,0.535744,nan,0.553821,nan
15,2020-01,DE,ANDROID_CONTROL,1726,0.537080,nan,0.568366,nan
20,2020-01,FR,ANDROID_CONTROL,2169,0.553711,nan,0.585984,nan
...,...,...,...,...,...,...,...,...
788,2021-06,FR,IOS_SHOW,8141,0.296892,0.18%,0.296892,0.18%
793,2021-06,GB,IOS_SHOW,13168,0.316373,0.57%,0.316373,0.57%
798,2021-06,IT,IOS_SHOW,5532,0.298988,-1.24%,0.298988,-1.24%
803,2021-06,MX,IOS_SHOW,2914,0.385724,-0.53%,0.385724,-0.53%


# LTV
for each cohort, avg orders/gmv per user(including churned user (LTV = zero)) in the 30d, 90d, 180d, 360d. 

## Orders

In [59]:
ltvo_cols = []
for x in df.columns:
    if '_orders_' in x:
        ltvo_cols.append(x)
ltvo_cols  

['avg_orders_30d',
 'avg_orders_60d',
 'avg_orders_90d',
 'avg_orders_180d',
 'avg_orders_360d',
 'sd_orders_30d',
 'sd_orders_60d',
 'sd_orders_90d',
 'sd_orders_180d',
 'sd_orders_360d']

In [65]:
ltvo = df[df['bucket'].apply(lambda x: x != 'not_in_exp')].sort_values(['cohort', 'bucket'])\
    [['cohort', 'shipping_country', 'bucket', 'users']+ltvo_cols].copy(deep=True)
ltvo.tail()

,cohort,shipping_country,bucket,users,avg_orders_30d,avg_orders_60d,avg_orders_90d,avg_orders_180d,avg_orders_360d,sd_orders_30d,sd_orders_60d,sd_orders_90d,sd_orders_180d,sd_orders_360d
788,2021-06,FR,IOS_SHOW,8141,3.279573,3.490112,3.490112,3.490112,3.490112,4.176867,4.759573,4.759573,4.759573,4.759573
793,2021-06,GB,IOS_SHOW,13168,2.726458,2.938259,2.938259,2.938259,2.938259,4.246601,4.874915,4.874915,4.874915,4.874915
798,2021-06,IT,IOS_SHOW,5532,3.112075,3.326283,3.326283,3.326283,3.326283,4.495288,4.893258,4.893258,4.893258,4.893258
803,2021-06,MX,IOS_SHOW,2914,4.336651,4.770075,4.770075,4.770075,4.770075,6.749380,7.914695,7.914695,7.914695,7.914695
808,2021-06,US,IOS_SHOW,69124,3.937185,4.308605,4.308880,4.308880,4.308880,7.388659,8.559274,8.559444,8.559444,8.559444


### Calculate diff


In [66]:
for c in [
    'avg_orders_30d',
     'avg_orders_60d',
     'avg_orders_90d',
     'avg_orders_180d',
     'avg_orders_360d'
    ]:
    ltvo[c+'_diff'] = ltvo.apply(lambda x: show_control_diff(x, ltvo, c, diff_pct=False), axis=1)
    
temp = zip([
    'avg_orders_30d',
     'avg_orders_60d',
     'avg_orders_90d',
     'avg_orders_180d',
     'avg_orders_360d'
    ], [c+'_diff' for c in [
    'avg_orders_30d',
     'avg_orders_60d',
     'avg_orders_90d',
     'avg_orders_180d',
     'avg_orders_360d'
    ]])
temp_cols = [l1 for l2 in temp for l1 in l2]

ltvo[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols].head()

,cohort,shipping_country,bucket,users,avg_orders_30d,avg_orders_30d_diff,avg_orders_60d,avg_orders_60d_diff,avg_orders_90d,avg_orders_90d_diff,avg_orders_180d,avg_orders_180d_diff,avg_orders_360d,avg_orders_360d_diff
0,2020-01,BR,ANDROID_CONTROL,3621,3.432477,NaN,3.992543,NaN,4.442695,NaN,5.257387,NaN,6.235294,NaN
5,2020-01,CA,ANDROID_CONTROL,1012,4.252964,NaN,5.176877,NaN,5.912055,NaN,7.369565,NaN,8.973320,NaN
10,2020-01,CL,ANDROID_CONTROL,1217,3.076417,NaN,3.506984,NaN,3.755957,NaN,4.516023,NaN,5.216927,NaN
15,2020-01,DE,ANDROID_CONTROL,1726,4.966396,NaN,5.774623,NaN,6.386443,NaN,7.517381,NaN,9.119351,NaN
20,2020-01,FR,ANDROID_CONTROL,2169,4.766252,NaN,5.628861,NaN,6.299677,NaN,7.405717,NaN,8.936376,NaN


### t test

In [75]:

for c in ['orders_30d',
     'orders_60d',
     'orders_90d',
     'orders_180d',
     'orders_360d']:
    ltvo[c+'_pval'] = ltvo.apply(lambda x: \
                                 show_control_t_test(x, data=ltvo, \
                                                     avg_col='avg_'+c, \
                                                     std_col='sd_'+c, \
                                                     n_col='users', \
                                                     alternative='two-sided') , axis=1)

    ltvo['avg_'+c+'_diff2'] = ltvo.apply(lambda x: f"{round(x['avg_'+c+'_diff'], 2)}" + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip(['avg_orders_30d',
             'avg_orders_60d',
             'avg_orders_90d',
             'avg_orders_180d',
             'avg_orders_360d'], ['avg_'+c+'_diff2' for c in ['orders_30d',
                                                             'orders_60d',
                                                             'orders_90d',
                                                             'orders_180d',
                                                             'orders_360d']])#, [c+'_test' for c in ltvo_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

ltvo[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols]

,cohort,shipping_country,bucket,users,avg_orders_30d,avg_orders_30d_diff2,avg_orders_60d,avg_orders_60d_diff2,avg_orders_90d,avg_orders_90d_diff2,avg_orders_180d,avg_orders_180d_diff2,avg_orders_360d,avg_orders_360d_diff2
0,2020-01,BR,ANDROID_CONTROL,3621,3.432477,nan,3.992543,nan,4.442695,nan,5.257387,nan,6.235294,nan
5,2020-01,CA,ANDROID_CONTROL,1012,4.252964,nan,5.176877,nan,5.912055,nan,7.369565,nan,8.973320,nan
10,2020-01,CL,ANDROID_CONTROL,1217,3.076417,nan,3.506984,nan,3.755957,nan,4.516023,nan,5.216927,nan
15,2020-01,DE,ANDROID_CONTROL,1726,4.966396,nan,5.774623,nan,6.386443,nan,7.517381,nan,9.119351,nan
20,2020-01,FR,ANDROID_CONTROL,2169,4.766252,nan,5.628861,nan,6.299677,nan,7.405717,nan,8.936376,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,2021-06,FR,IOS_SHOW,8141,3.279573,-0.01,3.490112,-0.02,3.490112,-0.02,3.490112,-0.02,3.490112,-0.02
793,2021-06,GB,IOS_SHOW,13168,2.726458,-0.31,2.938259,-0.32,2.938259,-0.32,2.938259,-0.32,2.938259,-0.32
798,2021-06,IT,IOS_SHOW,5532,3.112075,0.11,3.326283,0.15,3.326283,0.15,3.326283,0.15,3.326283,0.15
803,2021-06,MX,IOS_SHOW,2914,4.336651,-0.01,4.770075,0.07,4.770075,0.07,4.770075,0.07,4.770075,0.07


## GMV

In [68]:
ltvg_cols = []
for x in df.columns:
    if '_gmv_' in x:
        ltvg_cols.append(x)
ltvg_cols  

['avg_gmv_30d',
 'avg_gmv_60d',
 'avg_gmv_90d',
 'avg_gmv_180d',
 'avg_gmv_360d',
 'sd_gmv_30d',
 'sd_gmv_60d',
 'sd_gmv_90d',
 'sd_gmv_180d',
 'sd_gmv_360d']

In [72]:
ltvg = df[df['bucket'].apply(lambda x: x!='not_in_exp')].sort_values(['cohort', 'bucket'])\
    [['cohort', 'shipping_country', 'bucket', 'users']+ltvg_cols].copy(deep=True)
ltvg.tail()

,cohort,shipping_country,bucket,users,avg_gmv_30d,avg_gmv_60d,avg_gmv_90d,avg_gmv_180d,avg_gmv_360d,sd_gmv_30d,sd_gmv_60d,sd_gmv_90d,sd_gmv_180d,sd_gmv_360d
788,2021-06,FR,IOS_SHOW,8141,51.088316,54.158388,54.158388,54.158388,54.158388,77.308421,84.336763,84.336763,84.336763,84.336763
793,2021-06,GB,IOS_SHOW,13168,40.518340,43.499734,43.499734,43.499734,43.499734,73.943690,83.187461,83.187461,83.187461,83.187461
798,2021-06,IT,IOS_SHOW,5532,51.851000,55.123279,55.123279,55.123279,55.123279,78.225808,88.536552,88.536552,88.536552,88.536552
803,2021-06,MX,IOS_SHOW,2914,43.891798,46.538786,46.538786,46.538786,46.538786,90.200030,94.975498,94.975498,94.975498,94.975498
808,2021-06,US,IOS_SHOW,69124,53.784901,58.949447,58.951582,58.951582,58.951582,142.624579,158.319521,158.320066,158.320066,158.320066


### calculate diff

In [73]:
for c in [
    'avg_gmv_30d',
     'avg_gmv_60d',
     'avg_gmv_90d',
     'avg_gmv_180d',
     'avg_gmv_360d'
    ]:
    ltvg[c+'_diff'] = ltvg.apply(lambda x: show_control_diff(x, ltvg, c, diff_pct=False), axis=1)
    
temp = zip([
    'avg_gmv_30d',
     'avg_gmv_60d',
     'avg_gmv_90d',
     'avg_gmv_180d',
     'avg_gmv_360d'
    ], [c+'_diff' for c in [
    'avg_gmv_30d',
     'avg_gmv_60d',
     'avg_gmv_90d',
     'avg_gmv_180d',
     'avg_gmv_360d'
    ]])
temp_cols = [l1 for l2 in temp for l1 in l2]

ltvg[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols].head()

,cohort,shipping_country,bucket,users,avg_gmv_30d,avg_gmv_30d_diff,avg_gmv_60d,avg_gmv_60d_diff,avg_gmv_90d,avg_gmv_90d_diff,avg_gmv_180d,avg_gmv_180d_diff,avg_gmv_360d,avg_gmv_360d_diff
0,2020-01,BR,ANDROID_CONTROL,3621,20.181169,NaN,24.100917,NaN,26.792345,NaN,32.089084,NaN,38.807859,NaN
5,2020-01,CA,ANDROID_CONTROL,1012,39.620645,NaN,49.438461,NaN,57.444236,NaN,74.881494,NaN,95.087440,NaN
10,2020-01,CL,ANDROID_CONTROL,1217,17.492251,NaN,20.468660,NaN,22.362513,NaN,27.493557,NaN,33.919638,NaN
15,2020-01,DE,ANDROID_CONTROL,1726,61.823804,NaN,70.898191,NaN,77.723735,NaN,91.546226,NaN,115.417038,NaN
20,2020-01,FR,ANDROID_CONTROL,2169,49.731058,NaN,59.221698,NaN,65.990925,NaN,78.700748,NaN,99.795819,NaN


### t test

In [74]:

for c in ['gmv_30d',
     'gmv_60d',
     'gmv_90d',
     'gmv_180d',
     'gmv_360d']:
    ltvg[c+'_pval'] = ltvg.apply(lambda x: \
                                 show_control_t_test(x, data=ltvg, \
                                                     avg_col='avg_'+c, \
                                                     std_col='sd_'+c, \
                                                     n_col='users', \
                                                     alternative='two-sided') , axis=1)

    ltvg['avg_'+c+'_diff2'] = ltvg.apply(lambda x: f"{round(x['avg_'+c+'_diff'], 2)}" + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip(['avg_gmv_30d',
             'avg_gmv_60d',
             'avg_gmv_90d',
             'avg_gmv_180d',
             'avg_gmv_360d'], ['avg_'+c+'_diff2' for c in ['gmv_30d',
                                                             'gmv_60d',
                                                             'gmv_90d',
                                                             'gmv_180d',
                                                             'gmv_360d']])#, [c+'_test' for c in ltvg_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

ltvg[['cohort', 'shipping_country', 'bucket', 'users']+temp_cols]

,cohort,shipping_country,bucket,users,avg_gmv_30d,avg_gmv_30d_diff2,avg_gmv_60d,avg_gmv_60d_diff2,avg_gmv_90d,avg_gmv_90d_diff2,avg_gmv_180d,avg_gmv_180d_diff2,avg_gmv_360d,avg_gmv_360d_diff2
0,2020-01,BR,ANDROID_CONTROL,3621,20.181169,nan,24.100917,nan,26.792345,nan,32.089084,nan,38.807859,nan
5,2020-01,CA,ANDROID_CONTROL,1012,39.620645,nan,49.438461,nan,57.444236,nan,74.881494,nan,95.087440,nan
10,2020-01,CL,ANDROID_CONTROL,1217,17.492251,nan,20.468660,nan,22.362513,nan,27.493557,nan,33.919638,nan
15,2020-01,DE,ANDROID_CONTROL,1726,61.823804,nan,70.898191,nan,77.723735,nan,91.546226,nan,115.417038,nan
20,2020-01,FR,ANDROID_CONTROL,2169,49.731058,nan,59.221698,nan,65.990925,nan,78.700748,nan,99.795819,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,2021-06,FR,IOS_SHOW,8141,51.088316,-2.15,54.158388,-2.94,54.158388,-2.94,54.158388,-2.94,54.158388,-2.94
793,2021-06,GB,IOS_SHOW,13168,40.518340,-0.55,43.499734,-0.36,43.499734,-0.36,43.499734,-0.36,43.499734,-0.36
798,2021-06,IT,IOS_SHOW,5532,51.851000,-2.5,55.123279,-1.47,55.123279,-1.47,55.123279,-1.47,55.123279,-1.47
803,2021-06,MX,IOS_SHOW,2914,43.891798,-9.0,46.538786,-8.55,46.538786,-8.55,46.538786,-8.55,46.538786,-8.55
